## Ingestão de Dados: Meios de Pagamentos

Vamos desenvolver toda a lógica da extração dos dados disponibilizados na camada Landing, manipulando e gravando no Delta Lake entre as camadas Bronze, Silver e Gold.

In [1]:

# IMPORTS AND LIBRARIES
import os
import boto3
from botocore.exceptions import ClientError
from datetime import datetime
import logging

# Configuração do logger
logger = logging.getLogger("minio_logger")
logger.setLevel(logging.INFO)


# Configurando o formato do log
formatter = logging.Formatter('%(asctime)s - %(levelname)s - %(message)s')
console_handler = logging.StreamHandler()
console_handler.setFormatter(formatter)
logger.addHandler(console_handler)


# PySpark Libraries
from pyspark.sql import SparkSession, DataFrame
from pyspark.sql.functions import col, explode, lit, from_json

from pyspark.sql.types import StructType, StructField, StringType, LongType, TimestampType


### 02. Criando variáveis globais e configurando Spark Session

In [2]:
# Variáveis Globais e de Ambiente para o Projeto.

os.environ["MINIO_KEY"] = "developer"
os.environ["MINIO_SECRET"] = "developer01"
os.environ["MINIO_ENDPOINT"] = "http://minio:9000"


# Na ausência do dbtuils ou do Metastore vou usar o boto para me ajudar
# a iteragir com o storage
s3_client = boto3.client(
    's3',
    endpoint_url = os.environ.get("MINIO_ENDPOINT"),
    aws_access_key_id = os.environ.get("MINIO_KEY"),
    aws_secret_access_key = os.environ.get("MINIO_SECRET")
)

bucket_name = "bank-databr"

# Paths Data Storage
root_path_dir = f"{bucket_name}"
landing_path_dir = f"{root_path_dir}/landing/bacen"
bronze_path_dir = f"{root_path_dir}/bronze"
silver_path_dir = f"{root_path_dir}/silver"

# Partição da tabela Bronze, Data de referência
dt_partition = datetime.now().strftime("%Y-%m-%d")

In [3]:

spark = SparkSession.builder \
                    .appName("MeiosDePagamentoBancoCentral") \
                    .config("spark.hadoop.fs.s3a.endpoint", os.environ["MINIO_ENDPOINT"]) \
                    .config("spark.hadoop.fs.s3a.access.key", os.environ["MINIO_KEY"]) \
                    .config("spark.hadoop.fs.s3a.secret.key", os.environ["MINIO_SECRET"]) \
                    .config("spark.hadoop.fs.s3a.path.style.access", "true") \
                    .config("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem") \
                    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
                    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog") \
                    .getOrCreate()


/opt/spark/bin/load-spark-env.sh: line 68: ps: command not found


:: loading settings :: url = jar:file:/opt/spark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /root/.ivy2/cache
The jars for the packages stored in: /root/.ivy2/jars
software.amazon.awssdk#s3 added as a dependency
org.apache.hadoop#hadoop-aws added as a dependency
io.delta#delta-spark_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-475f2364-f4c4-45ab-82b6-cdd48143126e;1.0
	confs: [default]
	found software.amazon.awssdk#s3;2.26.30 in central
	found software.amazon.awssdk#aws-xml-protocol;2.26.30 in central
	found software.amazon.awssdk#aws-query-protocol;2.26.30 in central
	found software.amazon.awssdk#protocol-core;2.26.30 in central
	found software.amazon.awssdk#sdk-core;2.26.30 in central
	found software.amazon.awssdk#annotations;2.26.30 in central
	found software.amazon.awssdk#http-client-spi;2.26.30 in central
	found software.amazon.awssdk#utils;2.26.30 in central
	found org.reactivestreams#reactive-streams;1.0.4 in central
	found org.slf4j#slf4j-api;1.7.36 in central
	found software.amazon.awssdk#metric

### 03. From Landing to Bronze

Aqui vamos Ler os arquivos no formato original, extrair alguns metadados, e gravar no formato padrão, Delta.

#### Funções Utilitárias - Storage Handler Functions

In [4]:

def check_table_exists_in_metastore(schema_name:str, table_name: str) -> bool:
    """ Verifica se a tabela já existe no schema indicado dentro do Catalógo """

    return spark._jsparkSession.catalog() \
                               .tableExists(f"{schema_name}.{table_name}")


def check_table_exists_by_location(s3_minio_client:boto3.client, destionation_table_path: str) -> bool:
    """ Verifica se a tabela já existe, usando o gerenciador do storage """

    # Vou adaptar usando o boto3 já que não tenho por aqui nem o dbutils nem um metastore para checkar via tableExists()
    
    # Extrai o bucket e o prefixo (path)
    bucket_name, path = destionation_table_path.replace("s3a://", "").split("/", 1)
    
    try:
        # Verifica se o diretório existe no S3 (se o prefixo da pasta tiver objetos)
        response = s3_minio_client.list_objects_v2(Bucket=bucket_name, Prefix=path, MaxKeys=1)
        return 'Contents' in response
        
    except ClientError as e:
        return False


def write_delta_table_by_location(df_input: DataFrame, destionation_table_path: str) -> None:
    """ Escreve a tabela Delta no Storage de forma particionada
        Se a tabela já existir realiza upsert na partição indicada
        Caso não exista primeiramente cria a tabela no modo 'append'

        Args:
            df_input: dataframe a ser gravado no storage
            destionation_table_path: local de destino no storage
    """

    if check_table_exists_by_location(s3_client, destionation_table_path): 

        print(f"Tabela já existe: escrevendo nova partição em:\n\t*{destionation_table_path}")

        df_input.write \
                .format("delta") \
                .mode("append") \
                .option("replaceWhere", f"partition = {dt_partition}") \
                .partitionBy("dt_partition") \
                .save(destionation_table_path)   
    else:
        
        print(f"Tabela ainda não existe, criando nova tabela em: \n\t*{destionation_table_path}")

        #TODO: atualizar para considerar o EvolutionSchema, option("mergeSchema": True)
        # No formato atual estou considerando somente como EnforceSchema.

        df_input.write \
                .format("delta") \
                .mode("append") \
                .partitionBy("dt_partition") \
                .save(destionation_table_path) 

   

#### 01. Meios de Pagamento Trimestral

Todos os Meios de Pagamentos visão Trimestral (Landing to Bronze)
Aqui vamos nos preocupar em:
- Ler os dados no formato original, mantendo o schema da fonte
- Capturar e adicionar alguns metadados sobre a versão, origem, formato, registros entre outros.
- Gravar na Bronze em formato padronizado, usando Delta.

In [5]:
print("01. Leitura da fonte de Dados")

data_source_file_path = "s3a://" + landing_path_dir + "/meios_pagamentos_trimestral/data_18_01_2025_17_39_57.json"
print(f"\t* Data Source Pagamentos Trimestrais: {data_source_file_path}")
# Poderíamos usar também o data_*_.json para capturar todos os arquivos na origem.


df_pagamentos_trimestral_raw = spark.read \
                                    .option("inferSchema", True) \
                                    .json(data_source_file_path)

print("\n* Schema Original do arquivo origem")
df_pagamentos_trimestral_raw.printSchema
df_pagamentos_trimestral_raw.show(n=1, vertical=True, truncate=False)


print("02. Agora vamos capturar alguns metadados")

# Por padrão o spark já captura e armazena alguns metadadados default no Dataframe na coluna oculta _metadata
print('\t* Explorando a coluna _metadata')
df_selected = df_pagamentos_trimestral_raw.select( "_metadata")
df_selected.show(n=1, vertical=True, truncate=False)

# Vou adicionar somente um campo a mais chamado: ingestion_engine: para marcar qual ferramenta ou fluxo foi o responsável pela ingestão
# No nosso caso sempre um notebook-python com a lib DLT, mas poderia ser o recurso nify, airbyte, glue enfim, assim eu marcaria o id do recurso.

df_pagamentos_trimestral_bronze = df_pagamentos_trimestral_raw.select(
                                                                    col("*"),
                                                                    col('_metadata.*')                                                                
                                                                ) \
                                                              .withColumn('ingestion_engine', lit('python_dlt')) \
                                                              .withColumn('dt_partition', lit(dt_partition))


df_pagamentos_trimestral_bronze.show(n=1, vertical=True, truncate=True)

01. Leitura da fonte de Dados
	* Data Source Pagamentos Trimestrais: s3a://bank-databr/landing/bacen/meios_pagamentos_trimestral/data_18_01_2025_17_39_57.json


25/01/21 00:30:24 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.



* Schema Original do arquivo origem


-RECORD 0-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

-RECORD 0---------------------------------------------------------------------------------------------------------------------------------------------------------------------
 _metadata | {s3a://bank-databr/landing/bacen/meios_pagamentos_trimestral/data_18_01_2025_17_39_57.json, data_18_01_2025_17_39_57.json, 19142, 0, 19142, 2025-01-18 20:40:39} 



-RECORD 0--------------------------------------
 @odata.context         | https://was-p.bcn... 
 value                  | [{2024-09-30, 152... 
 file_path              | s3a://bank-databr... 
 file_name              | data_18_01_2025_1... 
 file_size              | 19142                
 file_block_start       | 0                    
 file_block_length      | 19142                
 file_modification_time | 2025-01-18 20:40:39  
 ingestion_engine       | python_dlt           
 dt_partition           | 2025-01-21           



In [6]:

print("03. Gravando Dados na Bronze no Formato Delta")

schema_name = "db_bank_databr"
table_name = "b_pagamentos_trimestrais_bc"

destionation_table_path = f"s3a://{bronze_path_dir}/{table_name}"

write_delta_table_by_location(df_pagamentos_trimestral_bronze, destionation_table_path)

03. Gravando Dados na Bronze no Formato Delta
Tabela já existe: escrevendo nova partição em:
	*s3a://bank-databr/bronze/b_pagamentos_trimestrais_bc


---

#### 02. Meios de Pagamento Mensal

Todos os Meios de Pagamentos visão Mensal (Landing to Bronze)

In [7]:

print("01. Leitura da fonte de Dados")

data_source_file_path = "s3a://" + landing_path_dir + "/meios_pagamentos_mensal/data_18_01_2025_17_39_57.json"
print(f"\t* Data Source Pagamentos Mensais: {data_source_file_path}")
# Poderíamos usar também o data_*_.json para capturar todos os arquivos na origem.


df_pagamentos_mensal_raw = spark.read \
                                    .option("inferSchema", True) \
                                    .json(data_source_file_path)

print("\n* Schema Original do arquivo origem")
df_pagamentos_mensal_raw.printSchema
df_pagamentos_mensal_raw.show(n=1, vertical=True, truncate=False)


print("02. Agora vamos capturar alguns metadados")

# Por padrão o spark já captura e armazena alguns metadadados default no Dataframe na coluna oculta _metadata
print('\t* Explorando a coluna _metadata')
df_selected = df_pagamentos_mensal_raw.select( "_metadata")
df_selected.show(n=1, vertical=True, truncate=False)

# Vou adicionar somente um campo a mais chamado: ingestion_engine: para marcar qual ferramenta ou fluxo foi o responsável pela ingestão
# No nosso caso sempre um notebook-python com a lib DLT, mas poderia ser o recurso nify, airbyte, glue enfim, assim eu marcaria o id do recurso.

df_pagamentos_mensal_bronze = df_pagamentos_mensal_raw.select(
                                                                  col("*"),
                                                                  col('_metadata.*')                                                                
                                                           ) \
                                                      .withColumn('ingestion_engine', lit('python_dlt')) \
                                                      .withColumn('dt_partition', lit(dt_partition))


df_pagamentos_mensal_bronze.show(n=1, vertical=True, truncate=False)


01. Leitura da fonte de Dados
	* Data Source Pagamentos Mensais: s3a://bank-databr/landing/bacen/meios_pagamentos_mensal/data_18_01_2025_17_39_57.json

* Schema Original do arquivo origem
-RECORD 0-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [8]:


print("03. Gravando Dados na Bronze no Formato Delta")

schema_name = "db_bank_databr"
table_name = "b_pagamentos_mensal_bc"

destionation_table_path = f"s3a://{bronze_path_dir}/{table_name}"

write_delta_table_by_location(df_pagamentos_mensal_bronze, destionation_table_path)


03. Gravando Dados na Bronze no Formato Delta
Tabela já existe: escrevendo nova partição em:
	*s3a://bank-databr/bronze/b_pagamentos_mensal_bc


#### 03. Meio de Pagamento - Cartões Trimestral

Pagamentos com Cartão visão Trimestral (Landing to Bronze)


In [9]:

print("01. Leitura da fonte de Dados")

data_source_file_path = "s3a://" + landing_path_dir + "/cartoes_trimestral/data_18_01_2025_17_39_57.json"
print(f"\t* Data Source Pagamentos Mensais: {data_source_file_path}")
# Poderíamos usar também o data_*_.json para capturar todos os arquivos na origem.


df_cartoes_trimestral_raw = spark.read \
                                    .option("inferSchema", True) \
                                    .json(data_source_file_path)

print("\n* Schema Original do arquivo origem")
df_cartoes_trimestral_raw.printSchema
df_cartoes_trimestral_raw.show(n=1, vertical=True, truncate=False)


print("02. Agora vamos capturar alguns metadados")

# Por padrão o spark já captura e armazena alguns metadadados default no Dataframe na coluna oculta _metadata
print('\t* Explorando a coluna _metadata')
df_selected = df_cartoes_trimestral_raw.select( "_metadata")
df_selected.show(n=1, vertical=True, truncate=False)

# Vou adicionar somente um campo a mais chamado: ingestion_engine: para marcar qual ferramenta ou fluxo foi o responsável pela ingestão
# No nosso caso sempre um notebook-python com a lib DLT, mas poderia ser o recurso nify, airbyte, glue enfim, assim eu marcaria o id do recurso.

df_cartoes_trimestral_bronze = df_cartoes_trimestral_raw.select(
                                                                  col("*"),
                                                                  col('_metadata.*')                                                                
                                                           ) \
                                                      .withColumn('ingestion_engine', lit('python_dlt')) \
                                                      .withColumn('dt_partition', lit(dt_partition))


df_cartoes_trimestral_bronze.show(n=1, vertical=True, truncate=False)


01. Leitura da fonte de Dados
	* Data Source Pagamentos Mensais: s3a://bank-databr/landing/bacen/cartoes_trimestral/data_18_01_2025_17_39_57.json

* Schema Original do arquivo origem
-RECORD 0----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [10]:

print("03. Gravando Dados na Bronze no Formato Delta")

schema_name = "db_bank_databr"
table_name = "b_cartoes_trimestral_bc"

destionation_table_path = f"s3a://{bronze_path_dir}/{table_name}"

write_delta_table_by_location(df_cartoes_trimestral_bronze, destionation_table_path)

03. Gravando Dados na Bronze no Formato Delta
Tabela já existe: escrevendo nova partição em:
	*s3a://bank-databr/bronze/b_cartoes_trimestral_bc
